<a href="https://colab.research.google.com/github/Weizhuo-Zhang/c-_primer_plus/blob/master/chapter08_function/chapter_08_function_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第8章 函数探幽

---

## 8.5 函数模板

函数模板是通用的函数描述，也就是说，它们使用泛型来定义函数，其中的泛型可用具体的类型（如int或double）替换。通过将类型作为参数传递给模板，可使编译器生成该类型的函数。由于模板允许以泛型（而不是具体类型）的方式编写程序。因此有时也被称为通用编程。由于类型是用参数表示的，因此模板特性有时也被称为参数化类型(parameterized types)。

函数模板允许以任意类型的方式来定义函数。例如，可以这样建立一个交换模板：
```c
template <typename AnyType>
void Swap(AnyType &a, AnyType &b)
{
    AnyType temp;
    temp = a;
    a = b;
    b = temp;
}
```

第一行指出，要建立一个模板，并将类型命名为`AnyType`。关键字template 和 typename 是必需的，除非可以使用关键字`class`代替`typename`。另外必须使用尖括号。类型名可以任意选择(这里为`AnyType`)


#### 程序清单 8.11 funtemp.cpp
```c
//funtemp.cpp -- using a function template
#include <iostream>
// Function template prototype
template <typename T>          // or class T
void Swap(T &a, T &b);

int main()
{
    using namespace std;
    int i = 10;
    int j = 20;
    cout << "i, j = " << i << ", " << j << ".\n";
    cout << "Using compiler generated int swapper:\n";
    Swap(i, j);    // generates void Swap(int &, int &)
    cout << "Now i, j = " << i << ", " << j << ".\n";
    
    double x = 24.5;
    double y = 81.5;
    cout << "x, y = " << x << ", " << y << ".\n";
    cout << "Using compiler generated double swapper:\n";
    Swap(x, y);    // generates void Swap(double &, double &)
    cout << "Now x, y = " << x << ", " << y << ".\n";
    return 0;
}

// function template definition
template <typename T> // or class T
void Swap(T &a, T &b)
{
    T temp;           // temp a variable of type T
    temp = a;
    a = b;
    b = temp;
}
```

注意，模板函数不能缩短可执行程序。对于程序清单8.11，最终仍由两个独立的函数定义。更常见的情形是，将模板放在头文件中，并在需要使用模板的文件中包含头文件。

### 8.5.1 重载的模板